In [ ]:
pip install beautifulsoup4 requests pandas --upgrade


In [4]:
from bs4 import BeautifulSoup

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

data = {
    "Company": [],
    "Website": [],
    "Relevant": [],
    "Category": [],
    "Manufacturer": [],
    "Brand": [],
    "Distributor": [],
    "F&B": [],
    "Probiotics": [],
    "Fortification": [],
    "Gut Health": [],
    "Womens Health": [],
    "Cognitive Health": []
}


companies = [
    {"name": "Namaste India", "url": "https://www.namasteindiafoods.com/milk.html"},
    {"name": "AB Biotics", "url": "https://www.ab-biotics.com/"},
    {"name": "Aliga Microalgae Company", "url": "https://www.aliga.dk/"},
    {"name": "Zinereo Pharma", "url": "https://www.zinereopharma.com/en/"},
    {"name": "Nutravends", "url": "https://nutravends.com/"}
]

def scrape_website(company):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(company["url"], headers=headers)
        response.raise_for_status()  #  for request errors
        soup = BeautifulSoup(response.content, 'html.parser')

        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        f_and_b = "Yes" if "food" in soup.text.lower() and "beverage" in soup.text.lower() else "No"
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"

        category = "N/A"
        if "manufacturer" in soup.text.lower():
            category = "Bulk (Manufacturer)"
        elif "distributor" in soup.text.lower():
            category = "Bulk (Distributor)"
        elif "brand" in soup.text.lower():
            category = "Brand"
        else:
            category = "F&B"

        data["Company"].append(company["name"])
        data["Website"].append(company["url"])
        data["Relevant"].append("Yes")
        data["Category"].append(category)
        data["Manufacturer"].append(manufacturer)
        data["Brand"].append(brand)
        data["Distributor"].append(distributor)
        data["F&B"].append(f_and_b)
        data["Probiotics"].append(probiotics)
        data["Fortification"].append(fortification)
        data["Gut Health"].append(gut_health)
        data["Womens Health"].append(womens_health)
        data["Cognitive Health"].append(cognitive_health)

    except requests.exceptions.HTTPError as e:
        print(f"HTTP error processing {company['name']}: {e}")
    except Exception as e:
        print(f"Error processing {company['name']}: {e}")


for company in companies:
    scrape_website(company)
    time.sleep(2)

df = pd.DataFrame(data)

file_path = "company_data.csv"
if os.path.exists(file_path):

    df.to_csv(file_path, mode='a', header=False, index=False)
else:

    df.to_csv(file_path, index=False)

print("Data scraping completed. Results saved to 'company_data.csv'.")


Data scraping completed. Results saved to 'company_data.csv'.


In [22]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
from io import StringIO 

# function to fetch from a public Google Sheets document
def fetch_companies_from_public_sheet(sheet_url):
    csv_url = sheet_url.replace('/edit?usp=sharing', '/gviz/tq?tqx=out:csv')

    response = requests.get(csv_url)
    
    if response.status_code == 200:
   
        data = pd.read_csv(StringIO(response.content.decode('utf-8')))
        
     
        data.columns = data.columns.str.strip().str.replace('"', '')  # Clean column names
        data = data[['Company', 'Website']].dropna()  # Ensure we only get companies and websites
        
        companies = []
        for index, row in data.iterrows():
            company_info = {
                "name": row['Company'].strip(),
                "url": row['Website'].strip()
            }
            companies.append(company_info)

        return companies
    else:
        print(f"Error fetching data: {response.status_code}")
        return []

def scrape_company_data(company):
    url = company["url"]
    try:
        print(f"Accessing URL: {url}")  
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')
       
        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        relevant = "Yes"  
        category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
        
     
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
        
        return {
            "Company": company["name"],
            "Website": url,
            "Relevant": relevant,
            "Category": category,
            "Manufacturer": manufacturer,
            "Brand": brand,
            "Distributor": distributor,
            "F&B": "Yes",  
            "Probiotics": probiotics,
            "Fortification": fortification,
            "Gut Health": gut_health,
            "Womens Health": womens_health,
            "Cognitive Health": cognitive_health
        }
    except requests.exceptions.RequestException as e:
        print(f"Error processing {company['name']}: {e}")
        return None


sheet_url = "https://docs.google.com/spreadsheets/d/1RESyISt077FgGm9tz-UZyYSu4M_0mVCvGBCLhmzN-p8/edit?usp=sharing"


companies = fetch_companies_from_public_sheet(sheet_url)


results = []

print(f"Number of companies fetched: {len(companies)}")  


for company in companies:
    print(f"Processing {company['name']}...")  
    data = scrape_company_data(company)
    if data:
        results.append(data)

    time.sleep(2)

df = pd.DataFrame(results)


csv_file = 'company_data.csv'
if os.path.exists(csv_file):
    df.to_csv(csv_file, mode='a', header=False, index=False)  # Append mode
else:
    df.to_csv(csv_file, index=False)  # Write mode
    print(f"Column headers added to '{csv_file}'.")

print(f"Data scraping completed. Results saved to '{csv_file}'.")

def fetch_companies_from_public_sheet(sheet_url):
    # converting the Google Sheets URL to a downloadable CSV format
    csv_url = sheet_url.replace('/edit?usp=sharing', '/gviz/tq?tqx=out:csv')
    
    # Fetch the CSV data
    response = requests.get(csv_url)
    
    if response.status_code == 200:
        # Read the CSV data into a DataFrame
        data = pd.read_csv(StringIO(response.content.decode('utf-8')))
        
        
        data.columns = data.columns.str.strip().str.replace('"', '')  
        data = data[['Company', 'Website']].dropna()  
        
        
        print(f"Number of rows in fetched data: {len(data)}")  # Number of rows fetched
        print(data.head())  # Print first few rows for verification
        
        companies = []
        for index, row in data.iterrows():
            company_info = {
                "name": row['Company'].strip(),
                "url": row['Website'].strip()
            }
            companies.append(company_info)

        return companies
    else:
        print(f"Error fetching data: {response.status_code}")
        return []


Number of companies fetched: 1
Processing Nestle...
Accessing URL: http://www.nestle.com
Data scraping completed. Results saved to 'company_data.csv'.


In [23]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
from io import StringIO

def fetch_companies_from_public_sheet(sheet_url):
    csv_url = sheet_url.replace('/edit?usp=sharing', '/gviz/tq?tqx=out:csv')
    response = requests.get(csv_url)
    
    if response.status_code == 200:
        data = pd.read_csv(StringIO(response.content.decode('utf-8')))
        data.columns = data.columns.str.strip().str.replace('"', '')
        data = data[['Company', 'Website']].dropna()
        
        print(f"Number of rows in fetched data: {len(data)}")
        print(data.head())
        
        companies = []
        for index, row in data.iterrows():
            company_info = {
                "name": row['Company'].strip(),
                "url": row['Website'].strip()
            }
            companies.append(company_info)

        return companies
    else:
        print(f"Error fetching data: {response.status_code}")
        return []

def scrape_company_data(company):
    url = company["url"]
    try:
        print(f"Accessing URL: {url}")
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        relevant = "Yes"
        category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
        
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
        
        return {
            "Company": company["name"],
            "Website": url,
            "Relevant": relevant,
            "Category": category,
            "Manufacturer": manufacturer,
            "Brand": brand,
            "Distributor": distributor,
            "F&B": "Yes",
            "Probiotics": probiotics,
            "Fortification": fortification,
            "Gut Health": gut_health,
            "Womens Health": womens_health,
            "Cognitive Health": cognitive_health
        }
    except requests.exceptions.RequestException as e:
        print(f"Error processing {company['name']}: {e}")
        return None

sheet_url = "https://docs.google.com/spreadsheets/d/1RESyISt077FgGm9tz-UZyYSu4M_0mVCvGBCLhmzN-p8/edit?usp=sharing"

companies = fetch_companies_from_public_sheet(sheet_url)

results = []

print(f"Number of companies fetched: {len(companies)}")

for company in companies:
    print(f"Processing {company['name']}...")
    data = scrape_company_data(company)
    if data:
        results.append(data)
    time.sleep(2)

df = pd.DataFrame(results)

csv_file = 'company_data.csv'
if os.path.exists(csv_file):
    df.to_csv(csv_file, mode='a', header=False, index=False)
else:
    df.to_csv(csv_file, index=False)
    print(f"Column headers added to '{csv_file}'.")

print(f"Data scraping completed. Results saved to '{csv_file}'.")


Number of rows in fetched data: 1
  Company                Website
0  Nestle  http://www.nestle.com
Number of companies fetched: 1
Processing Nestle...
Accessing URL: http://www.nestle.com
Data scraping completed. Results saved to 'company_data.csv'.


In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
from io import StringIO


In [32]:
def fetch_companies_from_public_sheet(sheet_url):
    csv_url = sheet_url.replace('/edit?usp=sharing', '/gviz/tq?tqx=out:csv')
    response = requests.get(csv_url)
    
    if response.status_code == 200:
        raw_data = response.content.decode('utf-8')
        print("Raw CSV data:\n", raw_data)
        data = pd.read_csv(StringIO(raw_data))
        data.columns = data.columns.str.strip().str.replace('"', '')
        data = data[['Company', 'Website']].dropna()
        
        print(f"Number of rows in fetched data: {len(data)}")
        print(data.head())

        if len(data) == 0:
            print("No data available after cleaning.")
            return []

        companies = []
        for index, row in data.iterrows():
            company_info = {
                "name": row['Company'].strip(),
                "url": row['Website'].strip()
            }
            companies.append(company_info)

        print(f"Fetched companies: {companies}")
        return companies
    else:
        print(f"Error fetching data: {response.status_code}")
        return []


In [33]:
fetch_companies_from_public_sheet("https://docs.google.com/spreadsheets/d/1RESyISt077FgGm9tz-UZyYSu4M_0mVCvGBCLhmzN-p8/edit?usp=sharing")

"Company","Website","Relevant","Category","","Manufacturer","Brand","Distributor","F&B","","Probiotics","Fortification","","Gut Health","Womens Health","Cognitive Health"
"Nestle","http://www.nestle.com","","","","","","","","","","","","","",""
Raw CSV data:
 "Company","Website","Relevant","Category","","Manufacturer","Brand","Distributor","F&B","","Probiotics","Fortification","","Gut Health","Womens Health","Cognitive Health"
"Nestle","http://www.nestle.com","","","","","","","","","","","","","",""
Number of rows in fetched data: 1
  Company                Website
0  Nestle  http://www.nestle.com
Fetched companies: [{'name': 'Nestle', 'url': 'http://www.nestle.com'}]


[{'name': 'Nestle', 'url': 'http://www.nestle.com'}]

In [26]:
print(data)

{'Company': 'Nestle', 'Website': 'http://www.nestle.com', 'Relevant': 'Yes', 'Category': 'Brand', 'Manufacturer': 'No', 'Brand': 'Yes', 'Distributor': 'No', 'F&B': 'Yes', 'Probiotics': 'No', 'Fortification': 'No', 'Gut Health': 'No', 'Womens Health': 'No', 'Cognitive Health': 'No'}


In [27]:
def scrape_company_data(company):
    url = company["url"]
    try:
        print(f"Accessing URL: {url}")
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        relevant = "Yes"
        category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
        
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
        
        return {
            "Company": company["name"],
            "Website": url,
            "Relevant": relevant,
            "Category": category,
            "Manufacturer": manufacturer,
            "Brand": brand,
            "Distributor": distributor,
            "F&B": "Yes",
            "Probiotics": probiotics,
            "Fortification": fortification,
            "Gut Health": gut_health,
            "Womens Health": womens_health,
            "Cognitive Health": cognitive_health
        }
    except requests.exceptions.RequestException as e:
        print(f"Error processing {company['name']} ({url}): {e}")
        return None


In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1RESyISt077FgGm9tz-UZyYSu4M_0mVCvGBCLhmzN-p8/edit?usp=sharing"

companies = fetch_companies_from_public_sheet(sheet_url)

results = []

print(f"Number of companies fetched: {len(companies)}")

for company in companies:
    print(f"Processing {company['name']}...")
    data = scrape_company_data(company)
    if data:
        results.append(data)
    
    time.sleep(2)

df = pd.DataFrame(results)

csv_file = 'company_data.csv'
if os.path.exists(csv_file):
    df.to_csv(csv_file, mode='a', header=False, index=False)
else:
    df.to_csv(csv_file, index=False)
    print(f"Column headers added to '{csv_file}'.")

print(f"Data scraping completed. Results saved to '{csv_file}'.")


# APPROACH -2 : Trying to Continue without google spreadsheet, having a local dataset 

In [6]:
import pandas as pd
# using different encoding instead of utf-8
try:
    dataset = pd.read_csv("dataset.csv", encoding="ISO-8859-1")
    #cleaning th dataa
    dataset = dataset.drop(columns=['Unnamed: 2'], errors='ignore')
    print(dataset)
except Exception as e:
    print(e)


                company_name                            url
0                     Nestle                 www.nestle.com
1   Dr. Reddy's Laboratories               www.drreddys.com
2                       Coca       www.coca-colacompany.com
3                     Pfizer                 www.pfizer.com
4                    PepsiCo                www.pepsico.com
5          Johnson & Johnson                    www.jnj.com
6                     Danone                 www.danone.com
7                      Bayer                  www.bayer.com
8              General Mills           www.generalmills.com
9      GlaxoSmithKline (GSK)                    www.gsk.com
10                 Kelloggs               www.kelloggs.com
11               Merck & Co.                  www.merck.com
12                  Unilever               www.unilever.com
13                     Roche                  www.roche.com
14             Nestle Waters           www.nestlewaters.com
15                    Sanofi            

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

dataset = pd.read_csv("dataset.csv", encoding="ISO-8859-1")
dataset = dataset.drop(columns=['Unnamed: 2'], errors='ignore')

def ensure_url_scheme(df, url_column):
    for index, row in df.iterrows():
        url = row[url_column]
        if not url.startswith('http://'):
            url = 'http://' + url
        row[url_column] = url

ensure_url_scheme(dataset, 'url')

def scrape_company_data(company):
    url = company["url"]
    try:
        print(f"Accessing URL: {url}")
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        relevant = "Yes"
        category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
        
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
        
        return {
            "Company": company["company_name"],
            "Website": url,
            "Relevant": relevant,
            "Category": category,
            "Manufacturer": manufacturer,
            "Brand": brand,
            "Distributor": distributor,
            "F&B": "Yes",
            "Probiotics": probiotics,
            "Fortification": fortification,
            "Gut Health": gut_health,
            "Womens Health": womens_health,
            "Cognitive Health": cognitive_health
        }
    except requests.exceptions.RequestException as e:
        print(f"Error processing {company['company_name']}: {e}")
        return None

results = []

for index, company in dataset.iterrows():
    print(f"Processing {company['company_name']}...")
    data = scrape_company_data(company)
    if data:
        results.append(data)

df = pd.DataFrame(results)

csv_file = 'company_data.csv'
df.to_csv(csv_file, index=False)

print(f"Data scraping completed. Results saved to '{csv_file}'.")


Processing Nestle...
Accessing URL: http://www.nestle.com
Processing Dr. Reddy's Laboratories...
Accessing URL: http://www.drreddys.com
Processing Coca...
Accessing URL: http://www.coca-colacompany.com
Processing Pfizer...
Accessing URL: http://www.pfizer.com
Processing PepsiCo...
Accessing URL: http://www.pepsico.com
Error processing PepsiCo: 403 Client Error: Forbidden for url: https://www.pepsico.com/
Processing Johnson & Johnson...
Accessing URL: http://www.jnj.com
Processing Danone...
Accessing URL: http://www.danone.com
Processing Bayer...
Accessing URL: http://www.bayer.com
Error processing Bayer: 403 Client Error: Forbidden for url: http://www.bayer.com/
Processing General Mills...
Accessing URL: http://www.generalmills.com
Processing GlaxoSmithKline (GSK)...
Accessing URL: http://www.gsk.com
Processing Kelloggs...
Accessing URL: http://www.kelloggs.com


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

dataset = pd.read_csv("dataset.csv", encoding="ISO-8859-1")
dataset = dataset.drop(columns=['Unnamed: 2'], errors='ignore')

def ensure_url_scheme(df, url_column):
    for index, row in df.iterrows():
        url = row[url_column]
        if not (url.startswith('http://') or url.startswith('https://')):
            url = 'http://' + url
        row[url_column] = url

ensure_url_scheme(dataset, 'url')

def scrape_company_data(company):
    url = company["url"]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    attempts = 3
    for attempt in range(attempts):
        try:
            print(f"Accessing URL: {url} (Attempt {attempt + 1})")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
            brand = "Yes" if "brand" in soup.text.lower() else "No"
            distributor = "Yes" if "distributor" in soup.text.lower() else "No"
            relevant = "Yes"
            category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
            
            probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
            fortification = "Yes" if "fortified" in soup.text.lower() else "No"
            gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
            womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
            cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
            
            return {
                "Company": company["company_name"],
                "Website": url,
                "Relevant": relevant,
                "Category": category,
                "Manufacturer": manufacturer,
                "Brand": brand,
                "Distributor": distributor,
                "F&B": "Yes",
                "Probiotics": probiotics,
                "Fortification": fortification,
                "Gut Health": gut_health,
                "Womens Health": womens_health,
                "Cognitive Health": cognitive_health
            }

        except requests.exceptions.HTTPError as e:
            if response.status_code == 403:
                print(f"Access denied for {company['company_name']}: {e}. Retrying...")
            else:
                print(f"Error processing {company['company_name']}: {e}")
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"Error processing {company['company_name']}: {e}")
            return None
    return None

results = []

for index, company in dataset.iterrows():
    print(f"Processing {company['company_name']}...")
    data = scrape_company_data(company)
    if data:
        results.append(data)

df = pd.DataFrame(results)

csv_file = 'company_data.csv'
df.to_csv(csv_file, index=False)

print(f"Data scraping completed. Results saved to '{csv_file}'.")


ParserError: Error tokenizing data. C error: Expected 2 fields in line 24, saw 3


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

dataset = pd.read_csv("dataset.csv", encoding="ISO-8859-1", on_bad_lines='skip')
dataset = dataset.drop(columns=['Unnamed: 2'], errors='ignore')

def ensure_url_scheme(df, url_column):
    for index, row in df.iterrows():
        url = row[url_column]
        if not url.startswith('http://'):
            url = 'http://' + url
        row[url_column] = url

ensure_url_scheme(dataset, 'url')

def scrape_company_data(company):
    url = company["url"]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    attempts = 3
    for attempt in range(attempts):
        try:
            print(f"Accessing URL: {url} (Attempt {attempt + 1})")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
            brand = "Yes" if "brand" in soup.text.lower() else "No"
            distributor = "Yes" if "distributor" in soup.text.lower() else "No"
            relevant = "Yes"
            category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
            probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
            fortification = "Yes" if "fortified" in soup.text.lower() else "No"
            gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
            womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
            cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
            
            return {
                "Company": company["company_name"],
                "Website": url,
                "Relevant": relevant,
                "Category": category,
                "Manufacturer": manufacturer,
                "Brand": brand,
                "Distributor": distributor,
                "F&B": "Yes",
                "Probiotics": probiotics,
                "Fortification": fortification,
                "Gut Health": gut_health,
                "Womens Health": womens_health,
                "Cognitive Health": cognitive_health
            }

        except requests.exceptions.HTTPError as e:
            if response.status_code == 403:
                print(f"Access denied for {company['company_name']}: {e}. Retrying...")
            else:
                print(f"Error processing {company['company_name']}: {e}")
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"Error processing {company['company_name']}: {e}")
            return None
    return None

results = []

for index, company in dataset.iterrows():
    print(f"Processing {company['company_name']}...")
    data = scrape_company_data(company)
    if data:
        results.append(data)

df = pd.DataFrame(results)
csv_file = 'company_data.csv'
df.to_csv(csv_file, index=False)

print(f"Data scraping completed. Results saved to '{csv_file}'.")


Processing Nestle...
Accessing URL: http://www.nestle.com (Attempt 1)
Processing Dr. Reddy's Laboratories...
Accessing URL: http://www.drreddys.com (Attempt 1)
Processing Coca...
Accessing URL: http://www.coca-colacompany.com (Attempt 1)
Processing Pfizer...
Accessing URL: http://www.pfizer.com (Attempt 1)
Processing PepsiCo...
Accessing URL: http://www.pepsico.com (Attempt 1)
Processing Johnson & Johnson...
Accessing URL: http://www.jnj.com (Attempt 1)
Processing Danone...
Accessing URL: http://www.danone.com (Attempt 1)
Processing Bayer...
Accessing URL: http://www.bayer.com (Attempt 1)
Access denied for Bayer: 403 Client Error: Forbidden for url: https://www.bayer.com/. Retrying...
Accessing URL: http://www.bayer.com (Attempt 2)
Access denied for Bayer: 403 Client Error: Forbidden for url: https://www.bayer.com/. Retrying...
Accessing URL: http://www.bayer.com (Attempt 3)
Access denied for Bayer: 403 Client Error: Forbidden for url: https://www.bayer.com/. Retrying...
Processing Gen

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from bs4 import BeautifulSoup

options = Options()
options.headless = True
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

data = {
    "Company": [],
    "Website": [],
    "Relevant": [],
    "Category": [],
    "Manufacturer": [],
    "Brand": [],
    "Distributor": [],
    "F&B": [],
    "Probiotics": [],
    "Fortification": [],
    "Gut Health": [],
    "Womens Health": [],
    "Cognitive Health": []
}

companies = [
    {'name': 'bayer', 'url': 'https://www.bayer.com'},
    {'name': 'lilly', 'url': 'https://www.lilly.com'},
    {'name': 'abbvie', 'url': 'https://www.abbvie.com'},
    {'name': 'medline', 'url': 'https://www.medline.com'},
]

def scrape_website(company):
    try:
        driver.get(company["url"])
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        f_and_b = "Yes" if "food" in soup.text.lower() and "beverage" in soup.text.lower() else "No"
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"

        category = "N/A"
        if manufacturer == "Yes":
            category = "Bulk (Manufacturer)"
        elif distributor == "Yes":
            category = "Bulk (Distributor)"
        elif brand == "Yes":
            category = "Brand"
        else:
            category = "F&B"

        data["Company"].append(company["name"])
        data["Website"].append(company["url"])
        data["Relevant"].append("Yes")
        data["Category"].append(category)
        data["Manufacturer"].append(manufacturer)
        data["Brand"].append(brand)
        data["Distributor"].append(distributor)
        data["F&B"].append(f_and_b)
        data["Probiotics"].append(probiotics)
        data["Fortification"].append(fortification)
        data["Gut Health"].append(gut_health)
        data["Womens Health"].append(womens_health)
        data["Cognitive Health"].append(cognitive_health)

    except Exception as e:
        print(f"Error processing {company['name']}: {e}")

for company in companies:
    scrape_website(company)

df = pd.DataFrame(data)
df.to_csv("test.csv", index=False)
driver.quit()
print("Data scraping completed. Results saved to 'company_data.csv'.")


Data scraping completed. Results saved to 'company_data.csv'.


# Scraping for nON-error Prone Wsbites

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

dataset = pd.read_csv("dataset.csv", encoding="ISO-8859-1", on_bad_lines='skip')
dataset = dataset.drop(columns=['Unnamed: 2'], errors='ignore')

def ensure_url_scheme(df, url_column):
    for index, row in df.iterrows():
        url = row[url_column]
        if not url.startswith('https://'):
            url = 'https://' + url
        row[url_column] = url

ensure_url_scheme(dataset, 'url')

def scrape_company_data(company):
    url = company["url"]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    attempts = 3
    for attempt in range(attempts):
        try:
            print(f"Accessing URL: {url} (Attempt {attempt + 1})")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
            brand = "Yes" if "brand" in soup.text.lower() else "No"
            distributor = "Yes" if "distributor" in soup.text.lower() else "No"
            relevant = "Yes"
            category = "Bulk (Manufacturer)" if manufacturer == "Yes" else "Bulk (Distributor)" if distributor == "Yes" else "Brand"
            
            probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
            fortification = "Yes" if "fortified" in soup.text.lower() else "No"
            gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
            womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
            cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"
            
            return {
                "Company": company["company_name"],
                "Website": url,
                "Relevant": relevant,
                "Category": category,
                "Manufacturer": manufacturer,
                "Brand": brand,
                "Distributor": distributor,
                "F&B": "Yes",
                "Probiotics": probiotics,
                "Fortification": fortification,
                "Gut Health": gut_health,
                "Womens Health": womens_health,
                "Cognitive Health": cognitive_health
            }

        except requests.exceptions.HTTPError as e:
            if response.status_code == 403:
                print(f"Access denied for {company['company_name']}: {e}. Retrying...")
            else:
                print(f"Error processing {company['company_name']}: {e}")
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"Error processing {company['company_name']}: {e}")
            return None
    return None

results = []

for index, company in dataset.iterrows():
    print(f"Processing {company['company_name']}...")
    data = scrape_company_data(company)
    if data:
        results.append(data)

df = pd.DataFrame(results)

csv_file = 'company_data.csv'
df.to_csv(csv_file, index=False)

print(f"Data scraping completed. Results saved to '{csv_file}'.")


Processing Nestle...
Accessing URL: https://www.nestle.com (Attempt 1)
Processing Dr. Reddy's Laboratories...
Accessing URL: https://www.drreddys.com (Attempt 1)
Processing Coca...
Accessing URL: https://www.coca-colacompany.com (Attempt 1)
Processing Pfizer...
Accessing URL: https://www.pfizer.com (Attempt 1)
Processing PepsiCo...
Accessing URL: https://www.pepsico.com (Attempt 1)
Processing Johnson & Johnson...
Accessing URL: https://www.jnj.com (Attempt 1)
Processing Danone...
Accessing URL: https://www.danone.com (Attempt 1)
Processing General Mills...
Accessing URL: https://www.generalmills.com (Attempt 1)
Processing GlaxoSmithKline (GSK)...
Accessing URL: https://www.gsk.com (Attempt 1)
Processing Merck & Co....
Accessing URL: https://www.merck.com (Attempt 1)
Processing Unilever...
Accessing URL: https://www.unilever.com (Attempt 1)
Processing Roche...
Accessing URL: https://www.roche.com (Attempt 1)
Processing Nestle Waters...
Accessing URL: https://www.nestlewaters.com (Attemp

# Selenium for 403 Error sites

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

options = Options()
options.headless = False
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

data = {
    "Company": [],
    "Website": [],
    "Relevant": [],
    "Category": [],
    "Manufacturer": [],
    "Brand": [],
    "Distributor": [],
    "F&B": [],
    "Probiotics": [],
    "Fortification": [],
    "Gut Health": [],
    "Womens Health": [],
    "Cognitive Health": []
}

companies = [
    {'name': 'bayer', 'url': 'https://www.bayer.com'},
    {'name': 'lilly', 'url': 'https://www.lilly.com'},
    {'name': 'abbvie', 'url': 'https://www.abbvie.com'},
    {'name': 'medline', 'url': 'https://www.medline.com'},
]

def scrape_website(company):
    try:
        driver.get(company["url"])
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        manufacturer = "Yes" if "manufacturer" in soup.text.lower() else "No"
        brand = "Yes" if "brand" in soup.text.lower() else "No"
        distributor = "Yes" if "distributor" in soup.text.lower() else "No"
        f_and_b = "Yes" if "food" in soup.text.lower() and "beverage" in soup.text.lower() else "No"
        probiotics = "Yes" if "probiotic" in soup.text.lower() else "No"
        fortification = "Yes" if "fortified" in soup.text.lower() else "No"
        gut_health = "Yes" if "gut health" in soup.text.lower() else "No"
        womens_health = "Yes" if "women's health" in soup.text.lower() else "No"
        cognitive_health = "Yes" if "cognitive health" in soup.text.lower() else "No"

        category = "N/A"
        if manufacturer == "Yes":
            category = "Bulk (Manufacturer)"
        elif distributor == "Yes":
            category = "Bulk (Distributor)"
        elif brand == "Yes":
            category = "Brand"
        else:
            category = "F&B"

        data["Company"].append(company["name"])
        data["Website"].append(company["url"])
        data["Relevant"].append("Yes")
        data["Category"].append(category)
        data["Manufacturer"].append(manufacturer)
        data["Brand"].append(brand)
        data["Distributor"].append(distributor)
        data["F&B"].append(f_and_b)
        data["Probiotics"].append(probiotics)
        data["Fortification"].append(fortification)
        data["Gut Health"].append(gut_health)
        data["Womens Health"].append(womens_health)
        data["Cognitive Health"].append(cognitive_health)

    except Exception as e:
        print(f"Error processing {company['name']}: {e}")

for company in companies:
    scrape_website(company)
    time.sleep(random.uniform(2, 5))

df = pd.DataFrame(data)
df.to_csv("test.csv", index=False)
driver.quit()
print("Data scraping completed. Results saved to 'test.csv'.")


Data scraping completed. Results saved to 'test.csv'.


# Merging the Two result csv's

In [14]:
import pandas as pd

# loading datasets
company_data = pd.read_csv("company_data.csv")
test = pd.read_csv("test.csv")

# mergin the datasets
merged_data = pd.concat([company_data, test], ignore_index=True)

# Save the mergred dataset to a new csv files
merged_data.to_csv("FINAL-result.csv", index=False)

print("Data merging completed. Results saved to 'result.xlsx'.")


Data merging completed. Results saved to 'result.xlsx'.
